In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

In [2]:
data = pd.read_csv("../input/train.csv",dtype={"acoustic_data": np.float32, "time_to_failure": np.float32}) #data is of 8.6GB, so nrows needs to be set
data.head(10)
#acoustic_data=signal & time_to_failure=quaketime

,acoustic_data,time_to_failure
0,12.0,1.4691
1,6.0,1.4691
2,8.0,1.4691
3,5.0,1.4691
4,8.0,1.4691
5,8.0,1.4691
6,9.0,1.4691
7,7.0,1.4691
8,-5.0,1.4691
9,3.0,1.4691


In [3]:
#EDA # i'll take these features to train
data.describe()

,acoustic_data,time_to_failure
count,6.291455e+08,6.291455e+08
mean,7.371910e-01,4.477084e-01
std,9.818262e+00,2.612789e+00
min,-5.515000e+03,9.550396e-05
25%,2.000000e+00,2.625997e+00
50%,5.000000e+00,5.349798e+00
75%,7.000000e+00,8.173395e+00
max,5.444000e+03,1.610740e+01


In [4]:
data.shape

(629145480, 2)

In [5]:
#make learnable features (empty yet)
nrows=200000
sets = int(np.floor(data.shape[0] / nrows))

X_train = pd.DataFrame(index=range(sets),columns = ['mean','std','min-0quat','25quat','50quat','75quat','max-1quat'])
y_train = pd.DataFrame(index=range(sets),columns = ['time_to_failure'])

In [6]:
X_train.head()

,mean,std,min-0quat,25quat,50quat,75quat,max-1quat
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
y_train.head()

,time_to_failure
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [8]:
#filling the tables
for set in (range(sets)):   #tqdm(range(sets))
    x=data.iloc[set*nrows:set*nrows+nrows]       #s.iloc[:3] returns us the first 3 rows (since it treats 3 as a position) and s.loc[:3] returns us the first 8 rows (since it treats 3 as a label)
    y = x['time_to_failure'].values[-1]
    x = x['acoustic_data'].values
    #adding mean,max,std to their respective columns(features)
    X_train.loc[set,'mean']=np.mean(x)
    X_train.loc[set,'std']=np.std(x)
    X_train.loc[set,'min-0quat']=np.quantile(x,0.01)
    X_train.loc[set,'25quat']=np.quantile(x,0.25)
    X_train.loc[set,'50quat']=np.quantile(x,0.5)
    X_train.loc[set,'75quat']=np.quantile(x,0.75)
    X_train.loc[set,'max-1quat']=np.quantile(x,0.99)
    y_train.loc[set,'time_to_failure'] = y
    

In [9]:
X_train.head(5)

,mean,std,min-0quat,25quat,50quat,75quat,max-1quat
0,4.83769,5.73268,-9,2,5,7,19
1,4.77767,6.09176,-11,2,5,7,21
2,4.94853,7.37905,-14,2,5,7,24
3,4.94923,6.70204,-12,3,5,7,23
4,4.86812,5.84906,-12,2,5,7,21


In [10]:
y_train.head(5)

,time_to_failure
0,1.4181
1,1.3659
2,1.3138
3,1.2617
4,1.2095


In [11]:
from sklearn.preprocessing import StandardScaler
#it will transform your data such that its distribution will have a mean value 0 and standard deviation of 1.
#Given the distribution of the data, each value in the dataset will have the sample mean value subtracted,
#and then divided by the standard deviation of the whole dataset.
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X_train)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


model=Sequential()
model.add(Dense(32,input_shape=(7,),activation = "relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(1))
adam=Adam(lr=0.0001)
model.compile(loss="mae",optimizer=adam)

model.fit(X_scaled,y_train,epochs = 500,validation_split=0.2)#batch_size

Train on 2516 samples, validate on 629 samples
Epoch 1/500
2516/2516 [==============================] - 3s 1ms/step - loss: 5.5664 - val_loss: 5.8672
Epoch 2/500
2516/2516 [==============================] - 0s 133us/step - loss: 5.3855 - val_loss: 5.6437
Epoch 3/500
2516/2516 [==============================] - 0s 131us/step - loss: 5.1776 - val_loss: 5.3473
Epoch 4/500
2516/2516 [==============================] - 0s 131us/step - loss: 4.9145 - val_loss: 4.9408
Epoch 5/500
2516/2516 [==============================] - 0s 131us/step - loss: 4.5588 - val_loss: 4.3718
Epoch 6/500
2516/2516 [==============================] - 0s 128us/step - loss: 4.0880 - val_loss: 3.6928
Epoch 7/500
2516/2516 [==============================] - 0s 129us/step - loss: 3.5703 - val_loss: 3.0874
Epoch 8/500
2516/2516 [==============================] - 0s 138us/step - loss: 3.0448 - val_loss: 2.7395
Epoch 9/500
2516/2516 [==============================] - 0s 140us/step - loss: 2.6389 - val_loss: 2.6313
Epoch 10/5

Epoch 78/500
2516/2516 [==============================] - 0s 143us/step - loss: 2.0366 - val_loss: 2.5175
Epoch 79/500
2516/2516 [==============================] - 0s 138us/step - loss: 2.0358 - val_loss: 2.5238
Epoch 80/500
2516/2516 [==============================] - 0s 146us/step - loss: 2.0358 - val_loss: 2.5182
Epoch 81/500
2516/2516 [==============================] - 0s 144us/step - loss: 2.0350 - val_loss: 2.5144
Epoch 82/500
2516/2516 [==============================] - 0s 140us/step - loss: 2.0352 - val_loss: 2.5194
Epoch 83/500
2516/2516 [==============================] - 0s 140us/step - loss: 2.0358 - val_loss: 2.5273
Epoch 84/500
2516/2516 [==============================] - 0s 141us/step - loss: 2.0347 - val_loss: 2.5229
Epoch 85/500
2516/2516 [==============================] - 0s 140us/step - loss: 2.0349 - val_loss: 2.5271
Epoch 86/500
2516/2516 [==============================] - 0s 140us/step - loss: 2.0345 - val_loss: 2.5203
Epoch 87/500
2516/2516 [======================

Epoch 155/500
2516/2516 [==============================] - 0s 142us/step - loss: 2.0237 - val_loss: 2.5350
Epoch 156/500
2516/2516 [==============================] - 0s 142us/step - loss: 2.0237 - val_loss: 2.5307
Epoch 157/500
2516/2516 [==============================] - 0s 143us/step - loss: 2.0265 - val_loss: 2.5315
Epoch 158/500
2516/2516 [==============================] - 0s 144us/step - loss: 2.0250 - val_loss: 2.5446
Epoch 159/500
2516/2516 [==============================] - 0s 141us/step - loss: 2.0228 - val_loss: 2.5356
Epoch 160/500
2516/2516 [==============================] - 0s 145us/step - loss: 2.0223 - val_loss: 2.5338
Epoch 161/500
2516/2516 [==============================] - 0s 144us/step - loss: 2.0231 - val_loss: 2.5419
Epoch 162/500
2516/2516 [==============================] - 0s 143us/step - loss: 2.0224 - val_loss: 2.5401
Epoch 163/500
2516/2516 [==============================] - 0s 136us/step - loss: 2.0232 - val_loss: 2.5423
Epoch 164/500
2516/2516 [============

2516/2516 [==============================] - 0s 143us/step - loss: 2.0170 - val_loss: 2.5597
Epoch 232/500
2516/2516 [==============================] - 0s 142us/step - loss: 2.0176 - val_loss: 2.5646
Epoch 233/500
2516/2516 [==============================] - 0s 145us/step - loss: 2.0167 - val_loss: 2.5522
Epoch 234/500
2516/2516 [==============================] - 0s 143us/step - loss: 2.0171 - val_loss: 2.5570
Epoch 235/500
2516/2516 [==============================] - 0s 138us/step - loss: 2.0159 - val_loss: 2.5526
Epoch 236/500
2516/2516 [==============================] - 0s 140us/step - loss: 2.0158 - val_loss: 2.5576
Epoch 237/500
2516/2516 [==============================] - 0s 143us/step - loss: 2.0163 - val_loss: 2.5550
Epoch 238/500
2516/2516 [==============================] - 0s 140us/step - loss: 2.0161 - val_loss: 2.5697
Epoch 239/500
2516/2516 [==============================] - 0s 137us/step - loss: 2.0167 - val_loss: 2.5653
Epoch 240/500
2516/2516 [==========================

2516/2516 [==============================] - 0s 138us/step - loss: 2.0107 - val_loss: 2.5918
Epoch 308/500
2516/2516 [==============================] - 0s 140us/step - loss: 2.0099 - val_loss: 2.5921
Epoch 309/500
2516/2516 [==============================] - 0s 139us/step - loss: 2.0103 - val_loss: 2.5922
Epoch 310/500
2516/2516 [==============================] - 0s 139us/step - loss: 2.0105 - val_loss: 2.5944
Epoch 311/500
2516/2516 [==============================] - 0s 143us/step - loss: 2.0105 - val_loss: 2.5883
Epoch 312/500
2516/2516 [==============================] - 0s 140us/step - loss: 2.0096 - val_loss: 2.5856
Epoch 313/500
2516/2516 [==============================] - 0s 142us/step - loss: 2.0110 - val_loss: 2.5943
Epoch 314/500
2516/2516 [==============================] - 0s 142us/step - loss: 2.0094 - val_loss: 2.5857
Epoch 315/500
2516/2516 [==============================] - 0s 138us/step - loss: 2.0095 - val_loss: 2.5876
Epoch 316/500
2516/2516 [==========================

2516/2516 [==============================] - 0s 141us/step - loss: 2.0055 - val_loss: 2.6246
Epoch 384/500
2516/2516 [==============================] - 0s 139us/step - loss: 2.0040 - val_loss: 2.6140
Epoch 385/500
2516/2516 [==============================] - 0s 143us/step - loss: 2.0036 - val_loss: 2.6226
Epoch 386/500
2516/2516 [==============================] - 0s 137us/step - loss: 2.0041 - val_loss: 2.6274
Epoch 387/500
2516/2516 [==============================] - 0s 138us/step - loss: 2.0037 - val_loss: 2.6235
Epoch 388/500
2516/2516 [==============================] - 0s 141us/step - loss: 2.0038 - val_loss: 2.6244
Epoch 389/500
2516/2516 [==============================] - 0s 137us/step - loss: 2.0061 - val_loss: 2.6160
Epoch 390/500
2516/2516 [==============================] - 0s 145us/step - loss: 2.0034 - val_loss: 2.6296
Epoch 391/500
2516/2516 [==============================] - 0s 138us/step - loss: 2.0034 - val_loss: 2.6234
Epoch 392/500
2516/2516 [==========================

2516/2516 [==============================] - 0s 141us/step - loss: 2.0004 - val_loss: 2.6521
Epoch 460/500
2516/2516 [==============================] - 0s 138us/step - loss: 1.9988 - val_loss: 2.6700
Epoch 461/500
2516/2516 [==============================] - 0s 141us/step - loss: 1.9996 - val_loss: 2.6594
Epoch 462/500
2516/2516 [==============================] - 0s 138us/step - loss: 2.0011 - val_loss: 2.6522
Epoch 463/500
2516/2516 [==============================] - 0s 139us/step - loss: 1.9998 - val_loss: 2.6538
Epoch 464/500
2516/2516 [==============================] - 0s 140us/step - loss: 1.9988 - val_loss: 2.6739
Epoch 465/500
2516/2516 [==============================] - 0s 136us/step - loss: 2.0000 - val_loss: 2.6725
Epoch 466/500
2516/2516 [==============================] - 0s 143us/step - loss: 1.9988 - val_loss: 2.6662
Epoch 467/500
2516/2516 [==============================] - 0s 144us/step - loss: 1.9991 - val_loss: 2.6718
Epoch 468/500
2516/2516 [==========================

In [14]:
submission_data = pd.read_csv('../input/sample_submission.csv',index_col = 'seg_id')
submission_data.tail(5)


,time_to_failure
seg_id,
seg_ff4236,0
seg_ff7478,0
seg_ff79d9,0
seg_ffbd6a,0
seg_ffe7cc,0


In [15]:
X_test = pd.DataFrame(columns = X_train.columns,dtype = np.float32,index=submission_data.index)
X_test.head(4)

,mean,std,min-0quat,25quat,50quat,75quat,max-1quat
seg_id,,,,,,,
seg_00030f,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seg_0012b5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seg_00184e,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seg_003339,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
from tqdm import tqdm
for seq in tqdm(X_test.index):
    test_data = pd.read_csv('../input/test/'+seq+'.csv')
    x = test_data['acoustic_data'].values
    X_test.loc[seq,'mean'] = np.mean(x)
    X_test.loc[seq,'std']  = np.std(x)
    X_test.loc[set,'min-0quat']=np.quantile(x,0.01)
    X_test.loc[set,'25quat']=np.quantile(x,0.25)
    X_test.loc[set,'50quat']=np.quantile(x,0.5)
    X_test.loc[set,'75quat']=np.quantile(x,0.75)
    X_test.loc[set,'max-1quat']=np.quantile(x,0.99)

100%|██████████| 2624/2624 [01:02<00:00, 41.79it/s]


In [32]:
X_test_scaled = scaler.transform(X_test)
#prediction
pred=model.predict(X_test_scaled)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [34]:
print(submission_data.shape)
submission_data.head()

(2624, 1)


,time_to_failure
seg_id,
seg_00030f,0
seg_0012b5,0
seg_00184e,0
seg_003339,0
seg_0042cc,0


In [63]:
print(pred.shape)
pred



(2625, 1)


array([[1.4410285],
       [1.4410285],
       [1.4410285],
       ...,
       [1.4410285],
       [1.4410285],
       [1.4410285]], dtype=float32)

In [66]:
submission_data['seg_id'] = submission_data.index


In [67]:
pred=submission_data["time_to_failure"]

In [ ]:
submission_data.to_csv('sub_earthquake.csv',index = False)